# OLS for price against actual demand generated (wind)

# Table of contents
 1.    [Necessary packages for OLS](#packages)
 <br><br>
 2.    [Data preparation](#dataprep)

       a. [Loading the dataset](#datainfo)
     
       b. [Removing NA values](#removena)
       
       c. [Filtering for uniformity](#filteruniform)
       <br>
 3.   [OLS Models](#olsmodel)
     
       a. [OLS Model (Ave MWH Reading)](#olsmodel1)
       
       b. [OLS Model ( $\frac{\text{Ave MWH Reading}}{\text{demand}}$ )](#olsmodel2)
       
       c. [OLS model ( $\text{Ave MWH Reading}+\frac{\text{Ave MWH Reading}}{\text{demand}}$ )](#olsmodel3)
       
       d. [OLS model ( $\log{\text{Ave MWH Reading}}$ )](#olsmodel4)

## A. Necessary packages  <a name="packages"></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## B. Data preparation <a name="dataprep"></a>

### 1. Loading the dataset <a name="datainfo"></a>

In [2]:
df = pd.read_csv('/Users/cececarino/Desktop/PE/Spot price forecasting/[Final] Datasets/NSW Wind(2022-2023).csv')

In [3]:
df.head()

,I,METER_DATA,GEN_DUID,1,DUID,MWH_READING,LASTCHANGED,INTERVAL,AveMWH_READING,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,D,METER_DATA,GEN_DUID,1.0,CULLRGWF,20.980000,2022/10/01 04:25:03,2022-10-01 04:30:00,11.805815,NSW1,2022-10-01 04:30:00,6274.24,152.14,TRADE
1,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,64.551765,2022/10/01 04:25:03,2022-10-01 04:30:00,11.805815,NSW1,2022-10-01 04:30:00,6274.24,152.14,TRADE
2,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,46.528053,2022/10/01 04:55:03,2022-10-01 05:00:00,10.212102,NSW1,2022-10-01 05:00:00,6417.10,156.11,TRADE
3,D,METER_DATA,GEN_DUID,1.0,CULLRGWF,18.160000,2022/10/01 04:55:03,2022-10-01 05:00:00,10.212102,NSW1,2022-10-01 05:00:00,6417.10,156.11,TRADE
4,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,42.270176,2022/10/01 05:25:03,2022-10-01 05:30:00,10.105626,NSW1,2022-10-01 05:30:00,6423.95,162.00,TRADE


### 2. Removing null values from the dataset <a name="removena"></a>

In [4]:
df.dropna(inplace=True)

### 3. Filtering data for uniformity <a name="filteruniform"></a>

- 2022-10-13 to 2023-11-21 timeframe as it was done for data on daylight saving hours

In [5]:
df['INTERVAL'] = pd.to_datetime(df['INTERVAL'])

# Filter the DataFrame for the time range from 8 am to 6 pm
start_date = pd.to_datetime('2022-10-13')
end_date = pd.to_datetime('2023-11-21')

df = df[(df['INTERVAL'] >= start_date) & (df['INTERVAL'] <= end_date)]

In [6]:
df.head()

,I,METER_DATA,GEN_DUID,1,DUID,MWH_READING,LASTCHANGED,INTERVAL,AveMWH_READING,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
1134,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,16.844007,2022/10/12 23:55:03,2022-10-13 00:00:00,13.577516,NSW1,2022-10-13 00:00:00,7286.84,157.17,TRADE
1135,D,METER_DATA,GEN_DUID,1.0,CULLRGWF,-0.040000,2022/10/12 23:55:03,2022-10-13 00:00:00,13.577516,NSW1,2022-10-13 00:00:00,7286.84,157.17,TRADE
1136,D,METER_DATA,GEN_DUID,1.0,CULLRGWF,-0.040000,2022/10/13 00:25:03,2022-10-13 00:30:00,12.832874,NSW1,2022-10-13 00:30:00,7115.44,161.42,TRADE
1137,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,14.808819,2022/10/13 00:25:03,2022-10-13 00:30:00,12.832874,NSW1,2022-10-13 00:30:00,7115.44,161.42,TRADE
1138,D,METER_DATA,GEN_DUID,1.0,CAPTL_WF,6.619242,2022/10/13 00:55:03,2022-10-13 01:00:00,13.288758,NSW1,2022-10-13 01:00:00,6885.86,158.43,TRADE


In [7]:
df.drop(columns=["DUID", "MWH_READING"], inplace=True)
df.drop_duplicates(inplace=True)
df.head()

,I,METER_DATA,GEN_DUID,1,LASTCHANGED,INTERVAL,AveMWH_READING,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
1134,D,METER_DATA,GEN_DUID,1.0,2022/10/12 23:55:03,2022-10-13 00:00:00,13.577516,NSW1,2022-10-13 00:00:00,7286.84,157.17,TRADE
1136,D,METER_DATA,GEN_DUID,1.0,2022/10/13 00:25:03,2022-10-13 00:30:00,12.832874,NSW1,2022-10-13 00:30:00,7115.44,161.42,TRADE
1138,D,METER_DATA,GEN_DUID,1.0,2022/10/13 00:55:03,2022-10-13 01:00:00,13.288758,NSW1,2022-10-13 01:00:00,6885.86,158.43,TRADE
1140,D,METER_DATA,GEN_DUID,1.0,2022/10/13 01:25:03,2022-10-13 01:30:00,14.584467,NSW1,2022-10-13 01:30:00,6586.08,148.99,TRADE
1142,D,METER_DATA,GEN_DUID,1.0,2022/10/13 01:55:03,2022-10-13 02:00:00,14.843988,NSW1,2022-10-13 02:00:00,6543.18,126.26,TRADE


## C. OLS Models <a name="olsmodel"></a>

### 1. OLS (Ave_MWH_Reading) <a name="olsmodel1"></a>

In [8]:
train_case, test_case = train_test_split(df, test_size=0.2, random_state=142)
print('Train shape: ', train_case.shape)
print('Test shape: ', test_case.shape)

Train shape:  (14714, 12)
Test shape:  (3679, 12)


In [9]:
reg_w = linear_model.LinearRegression()
X_trainw = train_case[['AveMWH_READING']]
y_trainw = train_case['RRP']

X_testw = test_case[['AveMWH_READING']]
y_testw = test_case['RRP']

resultsw = reg_w.fit(X_trainw, y_trainw)

In [10]:
predictedw = reg_w.predict(X_testw)
msew = ((np.array(y_testw)-predictedw)**2).sum()/len(y_testw)
rmsew = np.sqrt(msew)
print("reg MSE:", msew)
print("reg Root MSE:", rmsew)

reg MSE: 74010.88717523085
reg Root MSE: 272.04942046479505


In [11]:
X_trainw = sm.add_constant(X_trainw)  

modelw = sm.OLS(y_trainw, X_trainw)
resultsw = modelw.fit()

print(resultsw.summary())

                            OLS Regression Results                            
Dep. Variable:                    RRP   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     10.99
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           0.000919
Time:                        15:21:53   Log-Likelihood:            -1.0704e+05
No. Observations:               14714   AIC:                         2.141e+05
Df Residuals:                   14712   BIC:                         2.141e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            127.9111      7.675     16.

In [12]:
vifw = pd.DataFrame()
vifw["Variable"] = X_trainw.columns
vifw["VIF"] = [variance_inflation_factor(X_trainw.values, i) for i in range(X_trainw.shape[1])]

vifw

,Variable,VIF
0,const,7.108462
1,AveMWH_READING,1.000000


### 2. OLS ($\frac{\text{Average MWH Reading}}{\text{Demand}}$) <a name="olsmodel2"></a>

In [13]:
df2 = df.copy()
df2['MWHdemand'] = df2['AveMWH_READING']/df2['TOTALDEMAND']

In [14]:
df2.head()

,I,METER_DATA,GEN_DUID,1,LASTCHANGED,INTERVAL,AveMWH_READING,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE,MWHdemand
1134,D,METER_DATA,GEN_DUID,1.0,2022/10/12 23:55:03,2022-10-13 00:00:00,13.577516,NSW1,2022-10-13 00:00:00,7286.84,157.17,TRADE,0.001863
1136,D,METER_DATA,GEN_DUID,1.0,2022/10/13 00:25:03,2022-10-13 00:30:00,12.832874,NSW1,2022-10-13 00:30:00,7115.44,161.42,TRADE,0.001804
1138,D,METER_DATA,GEN_DUID,1.0,2022/10/13 00:55:03,2022-10-13 01:00:00,13.288758,NSW1,2022-10-13 01:00:00,6885.86,158.43,TRADE,0.001930
1140,D,METER_DATA,GEN_DUID,1.0,2022/10/13 01:25:03,2022-10-13 01:30:00,14.584467,NSW1,2022-10-13 01:30:00,6586.08,148.99,TRADE,0.002214
1142,D,METER_DATA,GEN_DUID,1.0,2022/10/13 01:55:03,2022-10-13 02:00:00,14.843988,NSW1,2022-10-13 02:00:00,6543.18,126.26,TRADE,0.002269


In [15]:
train_case, test_case = train_test_split(df2, test_size=0.2, random_state=142)
print('Train shape: ', train_case.shape)
print('Test shape: ', test_case.shape)

Train shape:  (14714, 13)
Test shape:  (3679, 13)


In [16]:
reg_2 = linear_model.LinearRegression()
X_train2 = train_case[['MWHdemand']]
y_train2 = train_case['RRP']

X_test2 = test_case[['MWHdemand']]
y_test2 = test_case['RRP']

results2 = reg_2.fit(X_train2, y_train2)

In [17]:
predicted2 = reg_2.predict(X_test2)
mse2 = ((np.array(y_test2)-predicted2)**2).sum()/len(y_test2)
rmse2 = np.sqrt(mse2)
print("reg MSE:", mse2)
print("reg Root MSE:", rmse2)

reg MSE: 73054.8791038009
reg Root MSE: 270.28666098015435


In [18]:
X_train2 = sm.add_constant(X_train2)  

model2 = sm.OLS(y_train2, X_train2)
results2 = model2.fit()

print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                    RRP   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     131.6
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           2.41e-30
Time:                        15:21:53   Log-Likelihood:            -1.0698e+05
No. Observations:               14714   AIC:                         2.140e+05
Df Residuals:                   14712   BIC:                         2.140e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        178.0196      7.034     25.309      0.0

### 3. OLS ($\text{AveMWH Reading} + \frac{\text{Average MWH Reading}}{\text{Demand}}$) <a name="olsmodel3"></a>

In [19]:
reg_3 = linear_model.LinearRegression()
X_train3 = train_case[['AveMWH_READING','MWHdemand']]
y_train3 = train_case['RRP']

X_test3 = test_case[['AveMWH_READING','MWHdemand']]
y_test3 = test_case['RRP']

results3 = reg_3.fit(X_train3, y_train3)

In [20]:
predicted3 = reg_3.predict(X_test3)
mse3 = ((np.array(y_test3)-predicted3)**2).sum()/len(y_test3)
rmse3 = np.sqrt(mse3)
print("reg MSE:", mse3)
print("reg Root MSE:", rmse3)

reg MSE: 71488.58195311476
reg Root MSE: 267.3734877528338


In [21]:
X_train3 = sm.add_constant(X_train3)  

model3 = sm.OLS(y_train3, X_train3)
results3 = model3.fit()

print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                    RRP   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     181.3
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           1.63e-78
Time:                        15:21:53   Log-Likelihood:            -1.0686e+05
No. Observations:               14714   AIC:                         2.137e+05
Df Residuals:                   14711   BIC:                         2.138e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            132.9101      7.590     17.

In [22]:
X_train3 = sm.add_constant(X_train3)

vif = pd.DataFrame()
vif["Variable"] = X_train3.columns
vif["VIF"] = [variance_inflation_factor(X_train3.values, i) for i in range(X_train3.shape[1])]

# Display the VIF values
print(vif)

         Variable       VIF
0           const  7.117249
1  AveMWH_READING  4.773285
2       MWHdemand  4.773285


### 4. OLS ($\log$ (Ave_MWH_Reading)) <a name="olsmodel4"></a>

In [23]:
df['AveMWH_READING_log'] = np.log(df['AveMWH_READING'])

In [24]:
train_case, test_case = train_test_split(df, test_size=0.2, random_state=142)
print('Train shape: ', train_case.shape)
print('Test shape: ', test_case.shape)

Train shape:  (14714, 13)
Test shape:  (3679, 13)


In [25]:
reg_log = linear_model.LinearRegression()
X_trainlog = train_case[['AveMWH_READING']]
y_trainlog = train_case['RRP']

X_testlog = test_case[['AveMWH_READING']]
y_testlog = test_case['RRP']

resultslog = reg_log.fit(X_trainlog, y_trainlog)

In [26]:
predictedlog = reg_log.predict(X_testlog)
mselog = ((np.array(y_testlog)-predictedlog)**2).sum()/len(y_testlog)
rmselog = np.sqrt(mselog)
print("reg MSE:", mselog)
print("reg Root MSE:", rmselog)

reg MSE: 74010.88717523085
reg Root MSE: 272.04942046479505


In [27]:
X_trainlog = sm.add_constant(X_trainlog)  

modellog = sm.OLS(y_trainlog, X_trainlog)
resultslog = modellog.fit()

print(resultslog.summary())

                            OLS Regression Results                            
Dep. Variable:                    RRP   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     10.99
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           0.000919
Time:                        15:21:54   Log-Likelihood:            -1.0704e+05
No. Observations:               14714   AIC:                         2.141e+05
Df Residuals:                   14712   BIC:                         2.141e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            127.9111      7.675     16.

In [28]:
viflog = pd.DataFrame()
viflog["Variable"] = X_trainlog.columns
viflog["VIF"] = [variance_inflation_factor(X_trainlog.values, i) for i in range(X_trainlog.shape[1])]

viflog

,Variable,VIF
0,const,7.108462
1,AveMWH_READING,1.000000
